In [12]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'2.1.0'

In [3]:
t = tf.range(0., 10.)
print(t)
t = t * 2
print(t)
t = t + 1
print(t)

tf.Tensor([0. 1. 2. 3. 4. 5. 6. 7. 8. 9.], shape=(10,), dtype=float32)
tf.Tensor([ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18.], shape=(10,), dtype=float32)
tf.Tensor([ 1.  3.  5.  7.  9. 11. 13. 15. 17. 19.], shape=(10,), dtype=float32)


In [4]:
@tf.function
def my_func(t):
    print("new graph created", t.dtype, t.shape)
    t = t * 2
    print(t)
    t = t + 1
    print(t)
    return t

t = tf.range(0., 10.)
my_func(t)

new graph created <dtype: 'float32'> (10,)
Tensor("mul:0", shape=(10,), dtype=float32)
Tensor("add:0", shape=(10,), dtype=float32)


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([ 1.,  3.,  5.,  7.,  9., 11., 13., 15., 17., 19.], dtype=float32)>

In [5]:
print(t.numpy())

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [6]:
my_func(tf.range(10., 20.))

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([21., 23., 25., 27., 29., 31., 33., 35., 37., 39.], dtype=float32)>

In [7]:
my_func(tf.range(10., 21.))
print("---")
my_func(tf.range(10, 20, dtype=tf.int32))

new graph created <dtype: 'float32'> (11,)
Tensor("mul:0", shape=(11,), dtype=float32)
Tensor("add:0", shape=(11,), dtype=float32)
---
new graph created <dtype: 'int32'> (10,)
Tensor("mul:0", shape=(10,), dtype=int32)
Tensor("add:0", shape=(10,), dtype=int32)


<tf.Tensor: shape=(10,), dtype=int32, numpy=array([21, 23, 25, 27, 29, 31, 33, 35, 37, 39], dtype=int32)>

In [8]:
import dadrah.playground.test_gradient_tape as tegrta

setGPU: Setting GPU to: 1


In [10]:
model = tegrta.make_model()
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dense_4 (Dense)              (None, 20)                40        
_________________________________________________________________
dense_5 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_6 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 21        
Total params: 901
Trainable params: 901
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
targets = np.arange(0.,10.)
predictions = np.random.random(size=10)*10
print(targets)
print(predictions)
print(targets-predictions)

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
[4.37055291 5.06897218 2.7622522  1.15019015 6.56934119 3.87367046
 2.58258601 3.62075818 5.2538975  5.83878212]
[-4.37055291 -4.06897218 -0.7622522   1.84980985 -2.56934119  1.12632954
  3.41741399  3.37924182  2.7461025   3.16121788]


In [15]:
print(tegrta.quantile_loss(targets, predictions))

tf.Tensor(
[0.43705529 0.40689722 0.07622522 1.66482886 0.25693412 1.01369659
 3.07567259 3.04131764 2.47149225 2.8450961 ], shape=(10,), dtype=float64)


In [16]:
print(tegrta.quantile_loss(targets, predictions))

tf.Tensor(
[0.43705529 0.40689722 0.07622522 1.66482886 0.25693412 1.01369659
 3.07567259 3.04131764 2.47149225 2.8450961 ], shape=(10,), dtype=float64)


In [17]:
optimizer = tf.keras.optimizers.Adam(0.005)

In [24]:
x = tf.random.uniform([300, 1], minval=1, maxval=3000, dtype=tf.float32)

In [25]:
x.shape

TensorShape([300, 1])

In [26]:
y = tf.random.uniform([300,1], minval=1, maxval=10, dtype=tf.float32)

In [28]:
train_dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(32)

In [33]:
epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:
            predictions = model(x_batch_train, training=True)
            loss_value = tegrta.quantile_loss(y_batch_train, predictions)
        
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        if step % 2 == 0:
            print("Training loss (for one batch) at step {}: {}".format(step, np.sum(loss_value)))
            print("Seen so far: %s samples" % ((step + 1) * 32))



Start of epoch 0
Training loss (for one batch) at step 0: 147.54522705078125
Seen so far: 32 samples
Training loss (for one batch) at step 2: 120.49897766113281
Seen so far: 96 samples
Training loss (for one batch) at step 4: 84.56854248046875
Seen so far: 160 samples
Training loss (for one batch) at step 6: 51.75752258300781
Seen so far: 224 samples
Training loss (for one batch) at step 8: 81.8460464477539
Seen so far: 288 samples

Start of epoch 1
Training loss (for one batch) at step 0: 52.03974914550781
Seen so far: 32 samples
Training loss (for one batch) at step 2: 56.4375114440918
Seen so far: 96 samples
Training loss (for one batch) at step 4: 63.15380096435547
Seen so far: 160 samples
Training loss (for one batch) at step 6: 52.665184020996094
Seen so far: 224 samples
Training loss (for one batch) at step 8: 42.150787353515625
Seen so far: 288 samples
